In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#원래 starter notebook에서는 tensorflow의 decision tree를
#(forest니까 이런 tree들의 모음인듯) 사용하는데, tensorflow는 버전 이슈로 지금
#사용하기는 힘드니까 틀을 베껴오고 평가 지표 유도 함수는 다른 걸 사용하자

dataset_df = pd.read_csv('train.csv')
dataset_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [2]:
dataset_df.describe() #얘는 꽤 의미있는 정보를 알려주니까 본격적인 분석 전에 확인하기

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [3]:
#Cabin column을 / 기준 3개로 쪼개고, 원래의 Cabin 없애기
dataset_df[['Deck', 'Cabin_num', 'Side']] = dataset_df['Cabin'].str.split('/', expand=True)
try:
    dataset_df = dataset_df.drop('Cabin', axis=1)
except KeyError:
    print("Field does not exist")

In [4]:
#모델 training에 필요하지 않은 column은 drop하기
dataset_df = dataset_df.drop(['PassengerId', 'Name'], axis=1)

In [5]:
#결측치를 찾고(sum까지), column마다 결측치가 얼마나 많은지 내림차순으로 정렬하여 확인
dataset_df.isnull().sum().sort_values(ascending=False)

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Deck            199
Cabin_num       199
Side            199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
Transported       0
dtype: int64

In [8]:
#결측치가 있는 행을 지우기에는 결측치가 꽤 많이 퍼져있는듯 하니 열마다 알맞은 값으로 결측치 채워넣기
#여기서 알맞은 값의 기준은 최빈값으로 하자 -> mode의 결과는 0행에 저장되기 때문에 iloc[0]까지 해야 정상적으로 작동
dataset_df=dataset_df.fillna(dataset_df.mode().iloc[0])
dataset_df['Cabin_num'] = dataset_df['Cabin_num'].astype(int)
dataset_df['VIP'] = dataset_df['VIP'].astype(int)
dataset_df['CryoSleep'] = dataset_df['CryoSleep'].astype(int)
#문자로 된 데이터는 forest에 넣을 수 없기 때문에 변환해야
from sklearn.preprocessing import LabelEncoder
train_enc = dataset_df.copy()
object_columns = train_enc.columns[train_enc.dtypes == 'object']
for o_col in object_columns:
    encoder = LabelEncoder()
    encoder.fit(train_enc[o_col])
    train_enc[o_col]=encoder.transform(train_enc[o_col])
train_enc

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Cabin_num,Side
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,0,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,0,98,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1499,1
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,6,1500,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,4,608,1


In [17]:
#실제 대회에서는 이런 식으로 무수히 많은 모델을 돌리던지/전처리 방식을 바꾸던지 해서 valid점수가 만족할 만큼 나오는 모델을 찾는다
#일단 여기서는 RandomForestClassifier만 활용해서 해보기
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
y = train_enc['Transported']
X = train_enc.drop('Transported', axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, random_state=1)

forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(X_train, y_train)
print("훈련 세트 점수: {:.3f}".format(forest.score(X_train, y_train)))
print("valid 세트 점수: {:.3f}".format(forest.score(X_valid, y_valid)))

훈련 세트 점수: 0.999
valid 세트 점수: 0.808


In [18]:
#제출하기 전 사전작업  
#test_df 불러오기
test_df = pd.read_csv('test.csv')
#얘도 모델 돌리려면 전처리 다 해줘야 한다
test_df[['Deck', 'Cabin_num', 'Side']] = test_df['Cabin'].str.split('/', expand=True)
try:
    test_df = test_df.drop('Cabin', axis=1)
except KeyError:
    print("Field does not exist")
test_df = test_df.drop(['PassengerId', 'Name'], axis=1)
test_df=test_df.fillna(test_df.mode().iloc[0])
test_df['Cabin_num'] = test_df['Cabin_num'].astype(int)
test_df['VIP'] = test_df['VIP'].astype(int)
test_df['CryoSleep'] = test_df['CryoSleep'].astype(int)
#문자로 된 데이터는 forest에 넣을 수 없기 때문에 변환해야
from sklearn.preprocessing import LabelEncoder
test_enc = test_df.copy()
object_columns = test_enc.columns[test_enc.dtypes == 'object']
for o_col in object_columns:
    encoder = LabelEncoder()
    encoder.fit(test_enc[o_col])
    test_enc[o_col]=encoder.transform(test_enc[o_col])
prediction = forest.predict(test_enc)

array([ True, False,  True, ...,  True,  True, False])

In [19]:
#제출하기: Dataframe으로 불러와서 prediction 내용 집어넣고 다시 csv로 바꾸기
sample_submission_df = pd.read_csv('sample_submission.csv')
sample_submission_df['Transported'] = prediction
sample_submission_df.to_csv('submission.csv', index=False)